In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

In [3]:
from langchain_groq import ChatGroq

groq_llm=ChatGroq(model="Llama3-8b-8192")

In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

hug_embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

USER_AGENT environment variable not set, consider setting it to identify your requests.
c:\Users\srish\anaconda3\envs\langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import bs4
loader=WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    )
)
web_loader=loader.load()

In [6]:
txt_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
web_chunks=txt_splitter.split_documents(web_loader)

faiss_db=FAISS.from_documents(web_chunks,hug_embeddings)
retriever=faiss_db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002301AAC1550>, search_kwargs={})

In [11]:
system_prompt=(
    """You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question.
    If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise
    \n\n
    {context}
    """
)

prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)

In [14]:
chain=create_stuff_documents_chain(groq_llm,prompt)
rag_chain=create_retrieval_chain(retriever,chain)

In [15]:
rag_chain.invoke({"input":"What is Self-Reflection"})

{'input': 'What is Self-Reflection',
 'context': [Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 3. Illustration of the Reflexion framework. (Image source: Shinn & Labash, 2023)\nThe heuristic function determines when the trajectory is inefficient or contains hallucination and should be stopped. Inefficient planning refers to trajectories that take too long without success. Hallucination is defined as encountering a sequence of consecutive identical actions that lead to the same observation in the environment.\nSelf-reflection is created by showing two-shot examples to LLM and each example is a pair of (failed trajectory, ideal reflection for guiding future changes in the plan). Then reflections are added into the agent’s working memory, up to three, to be used as context for querying LLM.'),
  Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Another quite distinct approach, LLM

In [17]:
rag_chain.invoke({"input":"How do we achieve it"})

{'input': 'How do we achieve it',
 'context': [Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.\nTask decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.'),
  Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated t

### Adding chat history

In [7]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

context_q_system_prompt=(
    "Given a chat history and the latest user question"
    "which might reference context in the chat history,"
    "formulate a standalone question which cab be understood"
    "wihtout the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

context_q_prompt=ChatPromptTemplate.from_messages(
    [
        ("system",context_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}")
    ]
)

In [8]:
history_aware_retriever=create_history_aware_retriever(groq_llm,retriever,context_q_prompt)
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002301AAC1550>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='A

In [12]:
qa_prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}")
    ]
)

In [13]:
qa_chain=create_stuff_documents_chain(groq_llm,qa_prompt)
rag_chains=create_retrieval_chain(history_aware_retriever,qa_chain)

In [16]:
from langchain_core.messages import AIMessage,HumanMessage

chat_history=[]
question="what is self-relfection"
res1=rag_chains.invoke({"input":question,"chat_history":chat_history})
print(res1['answer'])

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=res1['answer'])
    ]
)

question1="tell me more about it"
res2=rag_chains.invoke({"input":question1,"chat_history":chat_history})
print(res2['chat_history'])
print(res2['answer'])

According to the text, self-reflection is a process in autonomous agents that allows them to improve iteratively by refining past action decisions and correcting previous mistakes. It is a crucial aspect in real-world tasks where trial and error are inevitable.
[HumanMessage(content='what is self-relfection', additional_kwargs={}, response_metadata={}), AIMessage(content='According to the text, self-reflection is a process in autonomous agents that allows them to improve iteratively by refining past action decisions and correcting previous mistakes. It is a crucial aspect in real-world tasks where trial and error are inevitable.', additional_kwargs={}, response_metadata={})]
I don't have more information about self-reflection beyond what is provided in the text.


In [17]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id: str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

conversational_rag_chain=RunnableWithMessageHistory(
    rag_chains,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [19]:
conversational_rag_chain.invoke(
    {"input":"what is task decomposition?"},
    config={
        "configurable":{"session_id":"sri16"}
    },
)['answer']

'Task decomposition is the process of breaking down a complicated task into smaller, simpler steps. This is done to make the task more manageable and to allow for more efficient planning and execution.'

In [22]:
conversational_rag_chain.invoke(
    {"input":"what are the common ways of doing it?"},
    config={
        "configurable":{"session_id":"sri16"}
    },
)['answer']

'According to the provided context, the common ways to do task decomposition are:\n\n1. By LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?"\n2. By using task-specific instructions; e.g. "Write a story outline." for writing a novel\n3. With human inputs'